In [1]:
import os
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import gensim
from gensim import corpora
import string
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.externals import joblib  
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

data_train_path='GODJIA/datasets/20news-bydate/20news-bydate-train'
data_test_path='GODJIA/datasets/20news-bydate/20news-bydate-test'

/home/mis/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
/home/mis/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mis/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be rem

In [2]:
#将所有文件名读入
train_doc_names=[]
train_datas=[]
label_names=os.listdir(data_train_path)
train_label=[]
for i,label_name in enumerate(label_names):
    label_path=os.path.join(data_train_path,label_name)
    label_doc_names=os.listdir(label_path)
    train_label.extend([i]*len(label_doc_names))
    for label_doc_name in label_doc_names:
        label_doc_path=os.path.join(label_path,label_doc_name)
        train_doc_names.append(label_doc_path)
        with open(label_doc_path,'r',encoding="ISO-8859-1") as f:
            train_datas.append(f.read())
            f.close()
            
                
            
#print(train_doc_names)
print('训练文档总数：',len(train_datas))
print('训练标签总数：',len(train_label))
#print(train_datas[0])

训练文档总数： 11314
训练标签总数： 11314


In [3]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
textPre_FilePath='textPre.txt'
def textPrecessing(text):
    #小写化
    text = text.lower()
    #去除特殊标点
    for c in string.punctuation:
        text = text.replace(c, ' ')
    #分词
    wordLst = nltk.word_tokenize(text)
    #去除停用词
    filtered = [w for w in wordLst if w not in stopwords.words('english')]
    #仅保留名词或特定POS   
    refiltered =nltk.pos_tag(filtered)
    filtered = [w for w, pos in refiltered if pos.startswith('NN')]
    #词干化
    ps = PorterStemmer()
    filtered = [ps.stem(w) for w in filtered]

    return " ".join(filtered)

#train_datas_clean = [textPrecessing(doc) for doc in train_datas]

train_datas_clean = []
for i,desc in enumerate(train_datas) :
    train_datas_clean.append(textPrecessing(desc))
    if i%300==0:
        print(i)

with open(textPre_FilePath, 'w') as f:
    for line in train_datas_clean:
        f.write(line+"\n")



0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900
10200
10500
10800
11100


In [4]:
textPre_FilePath='textPre.txt'
train_datas_clean = []
with open(textPre_FilePath, 'r') as f:
    for line in f.readlines():
        if line != '':
            #train_datas_clean.append(line.strip())
            train_datas_clean.append(' '.join(line.strip().split()[7:-1]))
print(train_datas_clean[3])

articl geneva rutger hopper virginia edu lawrenc other letter doctrin hell noth preacher man god hell place need die god fact thing infinit die centuri concept centuri writer cours termonolog god death scriptur word see signifi death separ ceastat reason agoni cross time etern member godhead met ladi model saint year distanc form death wait home bid death separ death separ death separ fellowship outer dark go jesu hell respect man promis wise man older children guid life cours love harsh god danger choos rule conscienc matter wit thing caus fear jesu doubt pinprick man dealt error knowledg order correct faith faith knowledg jesu pinprick life scriptur heb priest sympath thing sin kenosi passag state attribut human medit mine thesi conscious christ peopl studi concluss jesu member triniti part human race knowledg grasp omnisci understand extent jesu man jesu women tens situat surpris event place time histori time discuss curcuit breaker brain info freethink organ use info organ duction 

In [5]:
tf_ModelPath='tf.model'
n_features=3500
#构建词汇统计向量并保存，仅运行首次
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(train_datas_clean)
joblib.dump(tf_vectorizer,tf_ModelPath )

['tf.model']

In [6]:
tf_ModelPath='tf.model'
#得到存储的tf_vectorizer,节省预处理时间
tf_vectorizer = joblib.load(tf_ModelPath)
tf = tf_vectorizer.transform(train_datas_clean)    

In [7]:
lda_path='lda.model'
lda = LatentDirichletAllocation(n_components=40, 
                                max_iter=50,
                                learning_method='batch')
lda.fit(tf) #tf即为Document_word Sparse Matrix  

#模型的保存/ 加载
joblib.dump(lda,lda_path )


['lda.model']

In [8]:
#得到存储的tf_vectorizer,节省预处理时间
lda = joblib.load(lda_path)
# 输出结果
def print_top_words(model, feature_names, n_top_words):
    print('打印每个主题下权重较高的term:')
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print('打印主题-词语分布矩阵:')
    print (model.components_)

n_top_words=20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


打印每个主题下权重较高的term:
Topic #0:
edu articl univers organ cs host cc news write washington cso depart scienc comput stanford anyon mail cwru illinoi distribut
Topic #1:
file window program problem use font error tape manag organ ms directori charact copi work version time sy disk util
Topic #2:
ac colorado ohio state event window magnu univers articl boulder organ valu time widget host way problem spot cs manag
Topic #3:
newsread version tin cleveland freenet van pl8 la reserv bnr tor chi que pit case organ bo det pl9 host
Topic #4:
imag color pictur jpeg gif qualiti compress bit display ticket rpi trial peopl format time edu viewer tank ifa gnv
Topic #5:
game year team season player time basebal fan leagu point play run win organ edu way flyer sport home seri
Topic #6:
ax max g9v b8f a86 pl giz bhj wm bxn gk sl bj qax m3 okz lj biz ql b8g
Topic #7:
cx w7 hz ah chz mv lk ck scx t7 pl a86 mc sc air sq w1 a7 b8 k8
Topic #8:
god peopl thing religion question life christian way evid time point 

In [9]:
train_set_path='train.set'
train_feature = lda.transform(tf)
print(train_feature)
print('训练集特征大小：',train_feature.shape)

train_set=[train_feature,train_label]
#print(train_set)
#训练集特征的保存/ 加载
joblib.dump(train_set,train_set_path )

[[  6.88705234e-05   6.88705234e-05   6.88705234e-05 ...,   6.88705234e-05
    6.88705234e-05   6.88705234e-05]
 [  3.08641975e-04   3.08641975e-04   3.08641975e-04 ...,   3.08641975e-04
    3.63102090e-02   3.08641975e-04]
 [  1.04166667e-03   1.04166667e-03   1.04166667e-03 ...,   1.04166667e-03
    1.04166667e-03   1.04166667e-03]
 ..., 
 [  4.54545455e-04   4.54545455e-04   4.54545455e-04 ...,   4.54545455e-04
    4.54545455e-04   4.88116809e-01]
 [  1.66666667e-03   1.66666667e-03   1.66666667e-03 ...,   1.66666667e-03
    1.66666667e-03   1.66666667e-03]
 [  8.33333333e-04   8.33333333e-04   8.33333333e-04 ...,   8.33333333e-04
    1.43931073e-01   8.33333333e-04]]
训练集特征大小： (11314, 40)


['train.set']

In [10]:
#得到存储的train_set,节省预处理时间
train_set_path='train.set'
train_set = joblib.load(train_set_path)
[train_feature,train_label]=train_set
print(train_feature)
print('训练集特征大小：',train_feature.shape)
print('训练标签总数：',len(train_label))

[[  6.88705234e-05   6.88705234e-05   6.88705234e-05 ...,   6.88705234e-05
    6.88705234e-05   6.88705234e-05]
 [  3.08641975e-04   3.08641975e-04   3.08641975e-04 ...,   3.08641975e-04
    3.63102090e-02   3.08641975e-04]
 [  1.04166667e-03   1.04166667e-03   1.04166667e-03 ...,   1.04166667e-03
    1.04166667e-03   1.04166667e-03]
 ..., 
 [  4.54545455e-04   4.54545455e-04   4.54545455e-04 ...,   4.54545455e-04
    4.54545455e-04   4.88116809e-01]
 [  1.66666667e-03   1.66666667e-03   1.66666667e-03 ...,   1.66666667e-03
    1.66666667e-03   1.66666667e-03]
 [  8.33333333e-04   8.33333333e-04   8.33333333e-04 ...,   8.33333333e-04
    1.43931073e-01   8.33333333e-04]]
训练集特征大小： (11314, 40)
训练标签总数： 11314


提取测试集特征

In [11]:
#将所有文件名读入
test_doc_names=[]
test_datas=[]
test_label_names=os.listdir(data_test_path)
test_label=[]
for i,label_name in enumerate(test_label_names):
    label_path=os.path.join(data_test_path,label_name)
    label_doc_names=os.listdir(label_path)
    test_label.extend([i]*len(label_doc_names))
    for label_doc_name in label_doc_names:
        label_doc_path=os.path.join(label_path,label_doc_name)
        test_doc_names.append(label_doc_path)
        with open(label_doc_path,'r',encoding="ISO-8859-1") as f:
            test_datas.append(f.read())
#print(test_doc_names)
print('测试文档总数：',len(test_doc_names))
print('测试标签总数：',len(test_label))
#print(test_datas[0])
#真正读取文档数据

测试文档总数： 7532
测试标签总数： 7532


In [12]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
test_textPre_FilePath='test_textPre.txt'
def textPrecessing(text):
    #小写化
    text = text.lower()
    #去除特殊标点
    for c in string.punctuation:
        text = text.replace(c, ' ')
    #分词
    wordLst = nltk.word_tokenize(text)
    #去除停用词
    filtered = [w for w in wordLst if w not in stopwords.words('english')]
    #仅保留名词或特定POS   
    refiltered =nltk.pos_tag(filtered)
    filtered = [w for w, pos in refiltered if pos.startswith('NN')]
    #词干化
    ps = PorterStemmer()
    filtered = [ps.stem(w) for w in filtered]

    return " ".join(filtered)

test_datas_clean = []
for i,desc in enumerate(test_datas) :
    test_datas_clean.append(textPrecessing(desc))
    if i%300==0:
        print(i)

with open(test_textPre_FilePath, 'w') as f:
    for line in test_datas_clean:
        f.write(line+"\n")


0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500


In [13]:
#读取预处理后的测试文本
test_textPre_FilePath='test_textPre.txt'
test_datas_clean = []
with open(test_textPre_FilePath, 'r') as f:
    for line in f.readlines():
        if line != '':
            test_datas_clean.append(' '.join(line.strip().split()[7:-1]))
print(test_datas_clean[3])

#读取tf_vectorizer，并对测试文本进行词频统计
tf_ModelPath='tf.model'
#得到存储的tf_vectorizer,节省预处理时间
tf_vectorizer = joblib.load(tf_ModelPath)
tf_test = tf_vectorizer.transform(test_datas_clean)    

earthquak predict organ cypress beaverton line articl atho rutger ingr write everyth heart citi portland countri state earthquak pass surpris surpris sleep mistak area fear area god misrepres god word word peopl word prophet bridg god suppos someon death glori power cloud lifetim year event question propheci make speaker prophet speak someth doubt condit relationship lord receiv propheci shadow doubt possibl imagin recognit wile follow way thing shadow doubt peopl check opinion realiti belief evid evid virtu ie faith luck result time christian exampl honesti backdrop lord prayer christian dan demonstr honesti start propheci conclud speaker


In [14]:
test_set_path='test.set'
test_feature = lda.transform(tf_test)
print(test_feature)
print('测试集特征大小：',test_feature.shape)

test_set=[test_feature,test_label]
#print(train_set)
#测试集特征的保存/ 加载
joblib.dump(test_set,test_set_path )

[[  4.16666667e-04   4.16666667e-04   4.16666667e-04 ...,   4.16666667e-04
    4.16666667e-04   4.16666667e-04]
 [  3.33333333e-04   3.33333333e-04   3.33333333e-04 ...,   3.33333333e-04
    1.66174794e-02   3.33333333e-04]
 [  6.09756098e-04   6.09756098e-04   6.09756098e-04 ...,   6.09756098e-04
    6.09756098e-04   6.09756098e-04]
 ..., 
 [  2.67202279e-02   2.93225645e-02   2.99643587e-02 ...,   2.11864407e-04
    2.11864407e-04   2.11864407e-04]
 [  2.55203207e-01   8.92857143e-04   1.43786483e-01 ...,   1.21120159e-01
    1.01320087e-01   8.92857143e-04]
 [  1.00000000e-03   1.00000000e-03   1.00000000e-03 ...,   5.39073133e-02
    1.00000000e-03   1.09744794e-01]]
测试集特征大小： (7532, 40)


['test.set']

In [15]:
#得到存储的test_set,节省预处理时间
test_set_path='test.set'
test_set = joblib.load(test_set_path)
[test_feature,test_label]=test_set
print(test_feature)
print('测试集特征大小：',test_feature.shape)
print('测试标签总数：',len(test_label))

[[  4.16666667e-04   4.16666667e-04   4.16666667e-04 ...,   4.16666667e-04
    4.16666667e-04   4.16666667e-04]
 [  3.33333333e-04   3.33333333e-04   3.33333333e-04 ...,   3.33333333e-04
    1.66174794e-02   3.33333333e-04]
 [  6.09756098e-04   6.09756098e-04   6.09756098e-04 ...,   6.09756098e-04
    6.09756098e-04   6.09756098e-04]
 ..., 
 [  2.67202279e-02   2.93225645e-02   2.99643587e-02 ...,   2.11864407e-04
    2.11864407e-04   2.11864407e-04]
 [  2.55203207e-01   8.92857143e-04   1.43786483e-01 ...,   1.21120159e-01
    1.01320087e-01   8.92857143e-04]
 [  1.00000000e-03   1.00000000e-03   1.00000000e-03 ...,   5.39073133e-02
    1.00000000e-03   1.09744794e-01]]
测试集特征大小： (7532, 40)
测试标签总数： 7532


训练集和测试集均处理完毕，开始分类器得训练，分类器选择SVM

In [16]:
def svm_c(x_train, x_test, y_train, y_test):
    # rbf核函数，设置数据权重
    svc = SVC(kernel='rbf', class_weight='balanced',)
    c_range = np.logspace(-5, 15, 11, base=2)
    gamma_range = np.logspace(-9, 3, 13, base=2)
    # 网格搜索交叉验证的参数范围，cv=3,3折交叉
    param_grid = [{'kernel': ['rbf'], 'C': c_range, 'gamma': gamma_range}]
    grid = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
    # 训练模型
    clf = grid.fit(x_train, y_train)
    # 计算测试集精度
    score = clf.score(x_test, y_test)
    print('精度为%s' % score)
    return clf

In [17]:
clf=svm_c(train_feature,test_feature,train_label,test_label)

精度为0.586165693043


In [18]:
#保存分类器
clf_Path='clf.model'
joblib.dump(clf,clf_Path)

['clf.model']

In [19]:
score=clf.score(test_feature, test_label)
print(score)

0.586165693043


In [20]:
score=clf.score(train_feature, train_label)
print(score)

0.760738907548
